# Libraries

In [1]:
# Core Libraries 
import pandas as pd
import numpy as np
import requests

# For Pattern
import re


# Datetime
from datetime import datetime

# For Yahoo Data Sets
import yfinance as yf

* Libraries Version 

In [2]:
print(f"Pandas Version: {pd.__version__} \n ----")
print(f"Numpy Version: {np.__version__}\n ----")
print(f"Re Version: {re.__version__}\n ----")
print(f"yfinance Version: {yf.__version__}\n ----")
print(f"Requests Version: {requests.__version__}")

Pandas Version: 2.0.2 
 ----
Numpy Version: 1.22.3
 ----
Re Version: 2.2.1
 ----
yfinance Version: 0.2.26
 ----
Requests Version: 2.31.0


## API KEY Configuration

* API keys hidden for security !!!

In [3]:
#for Api Key --- AlphaVantage Api Key  (https://www.alphavantage.co/)
from dotenv import load_dotenv
import os

def configure():
    load_dotenv()

# Formulas

In [4]:
Apple = yf.download("AAPL") 
Tesla = yf.download("TSLA")
SP500_index = yf.download("SPY") 

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


* Beta

In [5]:
SP500_data = SP500_index.copy()
Apple_data = Apple.copy()
Tesla_data = Tesla.copy()

SP500_2022 = SP500_data[(SP500_data.index >= "2021-12-31") & (SP500_data.index <= "2022-12-31")]
SP500_2022 = SP500_2022["Close"]

AAPL_2022 = Apple_data[(Apple_data.index >= "2021-12-31") & (Apple_data.index <= "2022-12-31")]
AAPL_2022 = AAPL_2022["Close"]

Tesla_2022 = Tesla_data[(Tesla_data.index >= "2021-12-31") & (Tesla_data.index <= "2022-12-31")]
Tesla_2022 = Tesla_2022["Close"]

# Calculate the daily returns for the stock and the market index
AAPL_returns  = AAPL_2022.pct_change()              # the return on an individual stock
Tesla_returns = Tesla_2022.pct_change()
SP500_returns = SP500_2022.pct_change()             # the return on an overall market

# Calculate the covariance between the stock returns and the market index returns
covariance   = AAPL_returns.cov(SP500_returns)
covariance_2 = Tesla_returns.cov(SP500_returns)

# Calculate the variance of the market index returns
variance   = SP500_returns.var()

# Calculate the beta coefficient
beta   = covariance / variance
beta_2 = covariance_2 / variance

print(f'Beta coefficient for Apple Inc.: {beta:.2f}')
print()
print(f'Beta coefficient for Tesla Company: {beta_2:.2f}')

Beta coefficient for Apple Inc.: 1.30

Beta coefficient for Tesla Company: 1.74


In [6]:
date_df = pd.DataFrame(pd.date_range(start='2017-12-31', end='2022-12-31', freq= "Y"))
date_list = date_df[0].dt.strftime("%Y-%m-%d")
date_list = date_list.to_list()

New_SP500 = {}
New_APPL  = {}
New_TSLA  = {}
New_APPL_returns = {}
New_TSLA_returns = {}
New_SP500_returns = {}
cov = {}
cov_2 = {}
var = {}
var = {}
Apple_Beta = []
Tesla_Beta = []
Beta = []
for i in np.arange(0,len(date_list)):
    z = i + 1
    
    if z < 6:
        x_SP    = SP500_data[(SP500_data.index >= f"{date_list[i]}") & (SP500_data.index <= f"{date_list[z]}")]
        y_AAPL  = Apple_data[(Apple_data.index >= f"{date_list[i]}") & (Apple_data.index <= f"{date_list[z]}")]
        k_TSLA  = Tesla_data[(Tesla_data.index >= f"{date_list[i]}") & (Tesla_data.index <= f"{date_list[z]}")]
        New_SP500[i] = pd.DataFrame(x_SP)
        New_APPL[i]  = pd.DataFrame(y_AAPL)
        New_TSLA[i]  = pd.DataFrame(k_TSLA)
    else:
        pass

for n in np.arange(0, 5):

    New_APPL[n]  = pd.DataFrame(New_APPL[n]["Close"])
    New_APPL_returns[n]  = pd.DataFrame(New_APPL[n]["Close"]).pct_change()

    New_TSLA[n]  = pd.DataFrame(New_TSLA[n]["Close"])
    New_TSLA_returns[n]  = pd.DataFrame(New_TSLA[n]["Close"]).pct_change()
    
    New_SP500[n] = pd.DataFrame(New_SP500[n]["Close"])
    New_SP500_returns[n] = pd.DataFrame(New_SP500[n]["Close"]).pct_change()


    New_APPL_returns[n][np.isnan(New_APPL_returns[n]) | np.isinf(New_APPL_returns[n])] = 0
    New_TSLA_returns[n][np.isnan(New_TSLA_returns[n]) | np.isinf(New_TSLA_returns[n])] = 0
    New_SP500_returns[n][np.isnan(New_SP500_returns[n]) | np.isinf(New_SP500_returns[n])] = 0
    
    cov[n] = np.cov(New_APPL_returns[n]["Close"].values, New_SP500_returns[n]["Close"].values)[0][1]
    cov_2[n] = np.cov(New_TSLA_returns[n]["Close"].values,New_SP500_returns[n]["Close"].values)[0][1]
    var[n] = np.var(New_SP500_returns[n]["Close"].values)

    x = cov[n] / var[n]
    y = cov_2[n] / var[n]

    Apple_Beta.append(x)
    Tesla_Beta.append(y)

Beta = pd.DataFrame(index = date_list[1:])
Beta["APPL Beta Coff."] = Apple_Beta
Beta["TSLA Beta Coff."] = Tesla_Beta
Beta

,APPL Beta Coff.,TSLA Beta Coff.
2018-12-31,1.258444,1.354869
2019-12-31,1.551936,1.329462
2020-12-31,1.159793,1.273195
2021-12-31,1.314310,1.872431
2022-12-31,1.304357,1.742990


** General Financial Formulas**

In [7]:
ticker = "AAPL"
FS = ["INCOME_STATEMENT","BALANCE_SHEET","CASH_FLOW"]   # Financial Statements

data = []
IS = {}                                                 # Income Statements
BS = {}                                                 # Balance Sheet
CF = {}                                                 # Cash Flow


for i in FS:
    url = f'https://www.alphavantage.co/query?function={i}&symbol={ticker}&apikey={os.getenv("Alphavantage_key")}'
    r = requests.get(url)
    data.append(r.json())

    data_df = pd.DataFrame(data)
    
    df = {}
    for i in np.arange(0, len(data_df)):
        df[i] = data_df.iloc[i].get("annualReports")

IS = pd.DataFrame(df[0]).T.reset_index()
BS = pd.DataFrame(df[1]).T.reset_index()
CF = pd.DataFrame(df[2]).T.reset_index()

FS = [IS,BS,CF]

IS_fiscal_Date = IS.iloc[0].to_list()
IS_index = IS.iloc[1].to_list()

FS_pattern = []

for i in np.arange(0, len(FS)):
    FS[i].replace("None",0,inplace = True)
    FS[i].fillna(0)

    FS[i].copy
    FS[i].loc[2:,[0,1,2,3,4]] = FS[i].loc[2:,[0,1,2,3,4]].astype("float64")
    
    FS[i].loc[:1] = IS_fiscal_Date
    FS[i].loc[1] = IS_index
    
    FS_pattern.append(FS[i]["index"])

pattern_list = pd.DataFrame()

IS = FS_pattern[0]
two = FS_pattern[1]
three = FS_pattern[2]

pat_list = [IS,two,three]

pattern = '[A-Z][^A-Z]*|[A-Z]*[^A-Z][^A-Z]*|[A-Z]*[^A-Z][^A-Z][^A-Z]*|[A-Z]*[^A-Z][^A-Z][^A-Z][^A-Z]*'

FS_0_join = []
for i in np.arange(0, len(IS)):
    FS_0_pattern = IS[i]
    FS_0_findall = re.findall(pattern,FS_0_pattern)
    FS_0_join.append(" ".join(FS_0_findall).title())
FS[0]["index"] = FS_0_join

FS_1_join = []
for z in np.arange(0, len(two)):
    FS_1_pattern = two[z]
    FS_1_findall = re.findall(pattern,FS_1_pattern)
    FS_1_join.append(" ".join(FS_1_findall).title())
FS[1]["index"] = FS_1_join

FS_2_join = []
for q in np.arange(0, len(three)):
    FS_2_pattern = three[q]
    FS_2_findall = re.findall(pattern,FS_2_pattern)
    FS_2_join.append(" ".join(FS_2_findall).title())
FS[2]["index"] = FS_2_join

index_names = ["INCOME STATEMENT","BALANCE SHEET","CASH FLOW"]

for i in np.arange(0,3):
    FS[i].columns = IS_fiscal_Date
    FS[i] = FS[i].drop(0)

    FS[i].index.name = index_names[i]

AAPL_FS = FS

In [8]:
TotalDebt = AAPL_FS[1][AAPL_FS[1]["fiscalDateEnding"] == "Current Debt"].values[0][1:] + AAPL_FS[1][AAPL_FS[1]["fiscalDateEnding"] == "Long Term Debt Noncurrent"].values[0][1:]

Current_liabilities   = AAPL_FS[1][AAPL_FS[1]["fiscalDateEnding"] == "Total Current Liabilities"].values[0][1:]
Current_asset         = AAPL_FS[1][AAPL_FS[1]["fiscalDateEnding"] == "Total Current Assets"].values[0][1:]
Total_Debt            = TotalDebt
Total_equity          = AAPL_FS[1][AAPL_FS[1]["fiscalDateEnding"] == "Total Shareholder Equity"].values[0][1:]
Long_term_debt        = AAPL_FS[1][AAPL_FS[1]["fiscalDateEnding"] == "Long Term Debt Noncurrent"].values[0][1:]
Net_income            = AAPL_FS[2][AAPL_FS[2]["fiscalDateEnding"] == "Net Income"].values[0][1:]
Dividends             = AAPL_FS[2][AAPL_FS[2]["fiscalDateEnding"] == "Dividend Payout"].values[0][1:]
Outstanding_share     = AAPL_FS[1][AAPL_FS[1]["fiscalDateEnding"] == "Common Stock Shares Outstanding"].values[0][1:]
Revenue               = AAPL_FS[0][AAPL_FS[0]["fiscalDateEnding"] == "Total Revenue"].values[0][1:]
Total_asset           = AAPL_FS[1][AAPL_FS[1]["fiscalDateEnding"] == "Total Assets"].values[0][1:]

NetWorkingCapital = []
CurrentRatio = []
DERatio  = []
FinancialLeverage = []
EPSRatio  = []
ROE = []
ProfitMargin  = []
AssetTurnOverRatio = []

for i in np.arange(0,len(Current_asset)):
    Net_Working_Capital     = Current_asset[i] - Current_liabilities[i]
    Current_Ratio           = float(Current_asset[i]) / float(Current_liabilities[i])
    DE_Ratio                = float(Total_Debt[i])    / float(Total_equity[i])
    Fianancial_Leverage     = float(Total_asset[i])    / float(Total_equity[i])
    EPS_Ratio               = ((Net_income[i] - Dividends[i]) / Outstanding_share[i])
    R_O_E                   = float(Net_income[i]) / float(Total_equity[i])
    Profit_Margin           = float(Net_income[i]) / float(Revenue[i])
    Asset_Turn_over_ratio   = float(Revenue[i]) / (float(Total_asset[i])/2)
    
    NetWorkingCapital.append(Net_Working_Capital)
    CurrentRatio.append(Current_Ratio)
    DERatio.append(DE_Ratio)
    FinancialLeverage.append(Fianancial_Leverage)
    EPSRatio.append(EPS_Ratio)
    ROE.append(R_O_E)
    ProfitMargin.append(Profit_Margin)
    AssetTurnOverRatio.append(Asset_Turn_over_ratio)


AAPL_Ratios = pd.DataFrame(NetWorkingCapital, columns = ["Net Working Capital"])
AAPL_Ratios = pd.DataFrame(AAPL_Ratios["Net Working Capital"])
AAPL_Ratios["Current Ratio"] = CurrentRatio
AAPL_Ratios["Current Ratio"] = AAPL_Ratios["Current Ratio"].map("{:,.2f}%".format)
AAPL_Ratios["D/E Ratio"] = DERatio
AAPL_Ratios["D/E Ratio"] = AAPL_Ratios["D/E Ratio"].map("{:,.2f}%".format)
AAPL_Ratios["EPS Ratio"] = EPSRatio
AAPL_Ratios["EPS Ratio"] = AAPL_Ratios["EPS Ratio"].map("{:,.2f}%".format)
AAPL_Ratios["Profit Margin"] = ProfitMargin
AAPL_Ratios["Profit Margin"] = AAPL_Ratios["Profit Margin"].map("{:,.2f}%".format)
AAPL_Ratios["Asset Turnover Ratio"] = AssetTurnOverRatio
AAPL_Ratios["Asset Turnover Ratio"] = AAPL_Ratios["Asset Turnover Ratio"].map("{:,.2f}%".format)
AAPL_Ratios["Financial Leverage Ratio"] = FinancialLeverage
AAPL_Ratios["Financial Leverage Ratio"] = AAPL_Ratios["Financial Leverage Ratio"].map("{:,.2f}%".format)
AAPL_Ratios["DuPont ROE"] = ROE
AAPL_Ratios["DuPont ROE"] = AAPL_Ratios["DuPont ROE"].map("{:,.2f}%".format)
AAPL_Ratios.index = AAPL_FS[0].columns.values[1:]

AAPL_Ratios.index.name = "Apple Inc."

AppleBeta = Apple_Beta[::-1]
AAPL_Ratios["Beta Coff."] = AppleBeta
AAPL_Ratios


,Net Working Capital,Current Ratio,D/E Ratio,EPS Ratio,Profit Margin,Asset Turnover Ratio,Financial Leverage Ratio,DuPont ROE,Beta Coff.
Apple Inc.,,,,,,,,,
2022-09-30,-1.857700e+10,0.88%,2.37%,5.33%,0.25%,2.22%,6.96%,1.97%,1.304357
2021-09-30,9.355000e+09,1.07%,1.98%,4.88%,0.26%,2.07%,5.56%,1.50%,1.314310
2020-09-30,3.832100e+10,1.36%,1.72%,2.55%,0.21%,1.68%,4.96%,0.88%,1.159793
2019-09-30,5.710100e+10,1.54%,1.19%,9.26%,0.22%,1.52%,3.74%,0.61%,1.551936
2018-09-30,1.541000e+10,1.13%,1.07%,9.64%,0.22%,1.45%,3.41%,0.56%,1.258444


### WACC (Weighted Average Cost of Capital)

[Referance](https://www.youtube.com/watch?v=yIJjUzyNGqg&ab_channel=Dividendology)

In [9]:
AAPL_Last_Stock_Price = yf.download("AAPL", period="min")
AAPL_Last_Stock_Price

[*********************100%***********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2023-09-08,178.350006,180.240005,177.789993,178.179993,178.179993,65551300


In [10]:
Market_value_of_equity = Outstanding_share[0] * AAPL_Last_Stock_Price["Close"].values[0]
print(f"Apple Company Market Capitalazation is {Market_value_of_equity}")

Apple Company Market Capitalazation is 2840799349726.8677


#### Cost of Debt

[Referance](https://analystanswers.com/total-debt-definition-formula-step-by-step-examples/)

In [11]:
Cost_of_debt = pd.DataFrame()
Cost_of_debt["Interest_expense"] = [AAPL_FS[0][AAPL_FS[0]["fiscalDateEnding"] == "Interest Expense"].values[0][1]]
Cost_of_debt["Total Debt"] = [TotalDebt[0]]
Cost_of_debt["Cost_debt"] = [(Cost_of_debt.iloc[0]["Interest_expense"] / Cost_of_debt.iloc[0]["Total Debt"])]
Cost_of_debt["Income_tax_expense"] = [AAPL_FS[0][AAPL_FS[0]["fiscalDateEnding"] == "Income Tax Expense"].values[0][1]]
Cost_of_debt["Before_Income_tax"] = [AAPL_FS[0][AAPL_FS[0]["fiscalDateEnding"] == "Income Before Tax"].values[0][1]] 
Cost_of_debt["Effective_tax_rate"]= Cost_of_debt["Income_tax_expense"] / Cost_of_debt["Before_Income_tax"]
Cost_of_debt["Cost_of_debt_after_tax"] = Cost_of_debt["Cost_debt"] * (1 - Cost_of_debt["Effective_tax_rate"])
Cost_of_debt 

,Interest_expense,Total Debt,Cost_debt,Income_tax_expense,Before_Income_tax,Effective_tax_rate,Cost_of_debt_after_tax
0,2.931000e+09,1.201980e+11,0.024385,1.930000e+10,1.191030e+11,0.162045,0.020433


#### Cost of Equity

In [12]:
SP500_index

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
1993-01-29,43.968750,43.968750,43.750000,43.937500,25.029364,1003200
1993-02-01,43.968750,44.250000,43.968750,44.250000,25.207390,480500
1993-02-02,44.218750,44.375000,44.125000,44.343750,25.260788,201300
1993-02-03,44.406250,44.843750,44.375000,44.812500,25.527815,529400
1993-02-04,44.968750,45.093750,44.468750,45.000000,25.634634,531500
...,...,...,...,...,...,...
2023-09-01,453.170013,453.670013,449.679993,451.190002,451.190002,58875700
2023-09-05,450.730011,451.059998,449.170013,449.239990,449.239990,55166200
2023-09-06,448.399994,448.510010,443.809998,446.220001,446.220001,70758500


In [13]:
SP_500 = SP500_index[SP500_index.index >= "2021-01-01"]
SP_500 = SP_500["Close"]
Apple_1  = Apple[Apple.index >= "2021-01-01"]
Apple_1  = Apple["Close"]

SP_500_r = SP_500.pct_change()
Apple_r  = Apple_1.pct_change()

cov = Apple_r.cov(SP_500_r) 
var = SP_500_r.var()

bet_a = cov / var
bet_a

Cost_of_Equity = pd.DataFrame()
Cost_of_Equity["Beta"] = [bet_a]
Cost_of_Equity["Market_return"] = [0.09]
Cost_of_Equity["Risk_Free_Rate"] = [yf.download("^TNX",period="10y").iloc[-1]["Close"]/100]
Cost_of_Equity["Cost_of_Equity"] = abs(Cost_of_Equity["Risk_Free_Rate"] + Cost_of_Equity["Beta"]*(Cost_of_Equity["Market_return"] - Cost_of_Equity["Risk_Free_Rate"]))
Cost_of_Equity

[*********************100%***********************]  1 of 1 completed


,Beta,Market_return,Risk_Free_Rate,Cost_of_Equity
0,1.28498,0.09,0.04258,0.103514


#### Weight of Debt and Equity

In [14]:
WACC = pd.DataFrame()
WACC["Total"] = [TotalDebt[0] + Market_value_of_equity]
WACC["W_Total Debt"] = Cost_of_debt["Total Debt"] / WACC["Total"]
WACC["W_MarketCap"]  = Market_value_of_equity / WACC["Total"]
WACC

,Total,W_Total Debt,W_MarketCap
0,2.960997e+12,0.040594,0.959406


In [15]:
wacc = Cost_of_debt["Cost_debt"]*WACC["W_Total Debt"]*(1-Cost_of_debt["Effective_tax_rate"]) + Cost_of_Equity["Cost_of_Equity"]*WACC["W_MarketCap"]
wacc = f"Weighted average cost of capital: {wacc}"
print(wacc)

Weighted average cost of capital: 0    0.100141
dtype: float64


### Bankruptcy Models Formulas

### Altman Z-Score

Altman Z-Score = 1.2A + 1.4B + 3.3C + 0.6D + 1.0E

Where:

A = working capital / total assets
B = retained earnings / total assets
C = earnings before interest and tax / total assets
D = market value of equity / total liabilities
E = sales /total assets

An Altman Z-score close to 0 suggests a company might be headed for bankruptcy, while a score closer to 3 suggests a company is in solid financial positioning.

In [16]:
Last_Stock_Price = Apple

Last_weekday = []
for i in np.arange(0,5):
    current_date = datetime.now().date()
    one_year_ago = current_date - pd.DateOffset(years=i)
    business_days = pd.bdate_range(end=one_year_ago, periods=7)
    last_friday = [business_days[business_days.weekday == 4].max()]

    Last_weekday.append(last_friday)

Last_weekday = pd.DataFrame(Last_weekday)
Last_weekday[0] = pd.DatetimeIndex(Last_weekday[0].dt.strftime("%Y-%m-%d"))
        
values = []
for i in np.arange(0,len(Last_weekday)):
    Data_prices = Last_Stock_Price["Close"][Last_Stock_Price.index == Last_weekday.iloc[i][0]]
    values.append(Data_prices)
values = pd.DataFrame(values)
        
df_without_nan = values.dropna(axis = 0, how= "all")
LastStockPrice = df_without_nan.values.flatten()
LastStockPrice = pd.DataFrame(LastStockPrice)
LastStockPrice = LastStockPrice.dropna()
LastStockPrice.index = Last_weekday[0]

LastStockPrice

,0
0,
2023-09-08,178.179993
2022-09-09,157.369995
2021-09-10,148.970001
2020-09-04,120.959999
2019-09-06,53.314999


In [17]:
GS_working_capital = AAPL_FS[1][AAPL_FS[1]["fiscalDateEnding"] == "Total Current Assets"].values[0][1] - AAPL_FS[1][AAPL_FS[1]["fiscalDateEnding"] == "Total Current Liabilities"].values[0][1]

A = GS_working_capital / AAPL_FS[1][AAPL_FS[1]["fiscalDateEnding"] == "Total Assets"].values[0][1]
 
B = AAPL_FS[1][AAPL_FS[1]["fiscalDateEnding"] == "Retained Earnings"].values[0][1] / AAPL_FS[1][AAPL_FS[1]["fiscalDateEnding"] == "Total Assets"].values[0][1]

C = AAPL_FS[0][AAPL_FS[0]["fiscalDateEnding"] == "Ebit"].values[0][1] / AAPL_FS[1][AAPL_FS[1]["fiscalDateEnding"] == "Total Assets"].values[0][1]

Market_value_of_equity_GS = AAPL_FS[1][AAPL_FS[1]["fiscalDateEnding"] == "Common Stock Shares Outstanding"].values[0][1] * LastStockPrice[0].values[0]

D = Market_value_of_equity_GS / AAPL_FS[1][AAPL_FS[1]["fiscalDateEnding"] == "Total Liabilities"].values[0][1]

Revenue = AAPL_FS[0][AAPL_FS[0]["fiscalDateEnding"] == "Net Interest Income"].values[0][1] + AAPL_FS[0][AAPL_FS[0]["fiscalDateEnding"] == "Non Interest Income"].values[0][1]

E = Revenue / AAPL_FS[1][AAPL_FS[1]["fiscalDateEnding"] == "Total Assets"].values[0][1]

AltmanZScore = pd.DataFrame()
AltmanZScore["A"] = [A]
AltmanZScore["B"] = [B]
AltmanZScore["C"] = [C]
AltmanZScore["D"] = [D]
AltmanZScore["E"] = [E]
AltmanZScore["Score"] = [(1.2*A) + (1.4*B) + (3.3*C) + (0.6*D) + (1.0*E)]

AltmanZScore.index.name = "Apple Inc."
AltmanZScore

,A,B,C,D,E,Score
Apple Inc.,,,,,,
0,-0.052663,-0.008697,0.345945,9.404036,1.109543,7.818214


#### Springate score

Springate score = 1.03A + 3.07B + .66C + .4D

A = Working capital / Total assets
B = EBIT / Total assets
C = Profit before tax / Current liabilities
D = Revenue / Total assets

In [18]:
X = AAPL_FS[0][AAPL_FS[0]["fiscalDateEnding"] == "Income Before Tax"].values[0][1] / AAPL_FS[1][AAPL_FS[1]["fiscalDateEnding"] == "Total Liabilities"].values[0][1]
Y = Revenue / AAPL_FS[1][AAPL_FS[1]["fiscalDateEnding"] == "Total Assets"].values[0][1]

Springate = pd.DataFrame()
Springate["A"] = [A]
Springate["B"] = [C]
Springate["C"] = [X]
Springate["D"] = [Y]
Springate["Score"] = [(1.03*A) + (3.07*C) + (0.66*X) + (0.4*Y)]

Springate

,A,B,C,D,Score
0,-0.052663,0.345945,0.394272,1.109543,1.711847


#### Zmijewski Score

Zmijewski Score = -4.336 - [4.513 * (Net Income / Total Assets)] + [5.679 * (Total Liabilities / Total Assets)] + [0.004 * (Current Assets / Current Liabilities)]

In [19]:
A_1 = AAPL_FS[0][AAPL_FS[0]["fiscalDateEnding"] == "Net Income"].values[0][1] / AAPL_FS[1][AAPL_FS[1]["fiscalDateEnding"] == "Total Assets"].values[0][1]
B_1 = AAPL_FS[1][AAPL_FS[1]["fiscalDateEnding"] == "Total Liabilities"].values[0][1] / AAPL_FS[1][AAPL_FS[1]["fiscalDateEnding"] == "Total Assets"].values[0][1]
C_1 = AAPL_FS[1][AAPL_FS[1]["fiscalDateEnding"] == "Total Current Assets"].values[0][1] / AAPL_FS[1][AAPL_FS[1]["fiscalDateEnding"] == "Total Current Liabilities"].values[0][1]

constant = -4.336
constant_2 = -4.513

ZM_score = pd.DataFrame()
ZM_score["4336"] = [constant]
ZM_score["4513"] = [C]
ZM_score["5679"] = [X]
ZM_score["0004"] = [Y]
ZM_score["Score"] = [(constant) + (constant_2*A_1) + (5.679*B_1) + (0.004 * C_1)]

ZM_score

,4336,4513,5679,0004,Score
0,-4.336,0.345945,0.394272,1.109543,-0.746089


In [32]:
Generaldf = pd.DataFrame()

Generaldf["Altman Z Score"] = [AltmanZScore["Score"].values[0]]
Generaldf["Springate Score"] = [Springate["Score"].values[0]]
Generaldf["Zmijewski Score"] = [ZM_score["Score"].values[0]]
Generaldf

,Altman Z Score,Springate Score,Zmijewski Score
0,7.818214,1.711847,-0.746089


# Volatility

In [20]:
df = yf.download('TSLA')
df

[*********************100%***********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2010-06-29,1.266667,1.666667,1.169333,1.592667,1.592667,281494500
2010-06-30,1.719333,2.028000,1.553333,1.588667,1.588667,257806500
2010-07-01,1.666667,1.728000,1.351333,1.464000,1.464000,123282000
2010-07-02,1.533333,1.540000,1.247333,1.280000,1.280000,77097000
2010-07-06,1.333333,1.333333,1.055333,1.074000,1.074000,103003500
...,...,...,...,...,...,...
2023-09-01,257.260010,259.079987,242.009995,245.009995,245.009995,132272500
2023-09-05,245.000000,258.000000,244.860001,256.489990,256.489990,129469600
2023-09-06,255.139999,255.389999,245.059998,251.919998,251.919998,116959800


In [21]:
df_close = df[["Close"]].copy()
df_close["CloseShift"] = df_close.shift()
df_close["PriceRelative"] = df_close["Close"] / df_close["CloseShift"]
df_close["Daily Return"] = np.log(df_close["PriceRelative"])
df_close = df_close.reset_index()

In [22]:
DailyVolatility = np.std(df_close['Daily Return'][1:])
print("The daily volatility of Tesla Company is: {:.2%}".format(DailyVolatility))

AnnualizedDailyVolatilityTradingDays = DailyVolatility*np.sqrt(252)
print("The annualized daily volatility of Tesla Company measured in trading days is: {:.2%}".format(AnnualizedDailyVolatilityTradingDays))

AnnualizedDailyVolatilityCalendarDays = DailyVolatility*np.sqrt(365)
print("The annualized daily volatility of Tesla Company measured in calendar days is: {:.2%}".format(AnnualizedDailyVolatilityCalendarDays))

The daily volatility of Tesla Company is: 3.59%
The annualized daily volatility of Tesla Company measured in trading days is: 56.93%
The annualized daily volatility of Tesla Company measured in calendar days is: 68.51%


### Weekly

In [23]:
weekly = pd.DataFrame(df_close.groupby([pd.Grouper(key="Date", freq="W")])["Close"].mean())
weekly["CloseShift"] = weekly["Close"].shift()
weekly["PriceRelative"] = weekly["Close"] / weekly["CloseShift"]
weekly["Weekly Return"] = np.log(weekly["PriceRelative"])

In [24]:
WeeklyVolatility = np.std(weekly['Weekly Return'][1:])
print("The weekly volatility of Tesla Company is: {:.2%}".format(WeeklyVolatility))

AnnualizedWeeklyVolatilityTradingWeeks = WeeklyVolatility*np.sqrt(52)
print("The annualized weekly volatility of Tesla Company measured in trading weeks is: {:.2%}".format(AnnualizedWeeklyVolatilityTradingWeeks))

The weekly volatility of Tesla Company is: 6.67%
The annualized weekly volatility of Tesla Company measured in trading weeks is: 48.07%


### Monthly

In [25]:
monthly = pd.DataFrame(df_close.groupby([pd.Grouper(key="Date", freq="M")])["Close"].mean())
monthly["CloseShift"] = monthly["Close"].shift()
monthly["PriceRelative"] = monthly["Close"] / monthly ["CloseShift"]
monthly["Monthly Return"] = np.log(monthly ["PriceRelative"])

In [26]:
MonthlyVolatility = np.std(monthly['Monthly Return'][1:])
print("The monthly volatility of Tesla Company is: {:.2%}".format(MonthlyVolatility))

AnnualizedMonthlyVolatility = MonthlyVolatility*np.sqrt(12)
print("The annualized monthly volatility of Tesla Company is: {:.2%}".format(AnnualizedMonthlyVolatility))

The monthly volatility of Tesla Company is: 14.25%
The annualized monthly volatility of Tesla Company is: 49.36%


### Annually

In [27]:
annually = pd.DataFrame(df_close.groupby([pd.Grouper(key="Date", freq="Y")])["Close"].mean())
annually = annually.dropna()

annually["CloseShift"] = annually["Close"].shift()
annually["PriceRelative"] = annually["Close"] / annually["CloseShift"]
annually["Annually Return"] = np.log(annually["PriceRelative"])

AnnualVolatility = np.std(annually['Annually Return'][1:])
print("The annual volatility of Tesla Company is: {:.2%}".format(AnnualVolatility))

The annual volatility of Tesla Company is: 57.23%


In [28]:
models = ['Annualized Daily Volatility by Trading Days','Annualized Daily Volatility by Calendar Days',
                'Annualized Weekly Volatility by Trading Weeks', 'Annualized Monthly Volatility',
                'Annual Volatility']
volatilities = [AnnualizedDailyVolatilityTradingDays,AnnualizedDailyVolatilityCalendarDays,
                AnnualizedWeeklyVolatilityTradingWeeks, AnnualizedMonthlyVolatility,
                AnnualVolatility]
new_volatilities = [f'{i*100:.2f}%' for i in volatilities]
compare_models = pd.DataFrame({ "Estimators": models, "Estimates": new_volatilities})
compare_models.sort_values(by = "Estimates", ascending = False)

,Estimators,Estimates
1,Annualized Daily Volatility by Calendar Days,68.51%
4,Annual Volatility,57.23%
0,Annualized Daily Volatility by Trading Days,56.93%
3,Annualized Monthly Volatility,49.36%
2,Annualized Weekly Volatility by Trading Weeks,48.07%
